# This notebook 
began as a duplication of the MODIS notebook for exploring chlorophyll data at the site of the OOI Cabled Array and was utilized as a generic explorer for MODIS Satellite Data

# MODIS Satellite Data - Map Projection


## Get Data

* [NASA Earth Observations (NEO)](https://neo.sci.gsfc.nasa.gov/view.php?datasetId=MY1DMW_CHLORA&date=2019-02-01) site
  - No bounding box
  - Can select monthly averages or 8-day
  - Clicking on `Download raw data` produces 30-45MB files at 0.1 deg resolution as NetCDF
* MODIS Chlorophyll Level 1 and 2 data are [here](http://oceancolor.gsfc.nasa.gov/cgi/browse.pl?sen=am)
* MODIS Chlorophyll Level 3 data are [here](http://oceancolor.gsfc.nasa.gov/cgi/l3)
* These data files will be saved onto your computers harddrive and must be moved into Pangeo in order to run these cells


## Set up Folders and Files
* In working Pangeo server, create a 'data' folder and a 'modis' subfolder. Upload your MODIS files to this subfolder. Do not capitalize folder names.
* Ensure that you have cloned a repo containing the 'golive_libary.py' file. It can be found in the robfatland/golive repo on GitHub.

## Data Presentation 

This notebook takes a look at how to superimpose MODIS chlorophyll data on a map and explore different methods of presentation


### Set up python environment
* Install all necessary packages

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’

# automatically reload external modules such as golive_library if they have changed
%load_ext autoreload
%autoreload 2

In [2]:
!pip install utm
!pip install pygeotools
!pip install tqdm
!pip install netCDF4
!pip install xarray
!pip install matplotlib
!pip install scipy
#!pip install dask
#!pip install utm

  Using cached pygeotools-0.6.0.tar.gz (80 kB)
  Created wheel for pygeotools: filename=pygeotools-0.6.0-py3-none-any.whl size=88056 sha256=3f3f0130ca6d1eba0757dfbb18cbaa15ba77593cdf045891f76ce0cd372a7947
  Stored in directory: /home/jovyan/.cache/pip/wheels/49/e7/5b/66b229126d8bd59141a175f0cd6ae71750ad87eb6e477620bc
Successfully built pygeotools
  Using cached tqdm-4.43.0-py2.py3-none-any.whl (59 kB)


In [3]:
%matplotlib inline

# Python Standard Library built-in modules
import sys
import os
import timeit
from datetime import timedelta, datetime
from random import randint as ri

# Third-party modules
from tqdm import tqdm
from matplotlib import pyplot as plt
import pandas as pd
import netCDF4
import xarray as xr
import dask
import numpy as np
from numpy import datetime64 as dt64, timedelta64 as td64
from scipy import stats
from scipy.signal import medfilt as mf
from ftplib import FTP
from ipywidgets import *                # interactivity
from traitlets import dlink             # interactivity
import utm
# from osgeo import gdal 
# from pygeotools.lib import iolib,timelib

# Local application/library specific imports
from golive_library import GoliveLibrary as g

In [ ]:
# this cell is for the section of the notebook beyond the intermezzo remarks
# verify the MODIS example file is present
#modis_d = g.set_data_directory('modis')
#image_d = g.set_data_directory('images')
#g.lsal(modis_d)

Open all files as an xarray dataset for processing 

In [4]:
fnm_field_index = 8                                                 # index of the filename after the ls line is split
m=[]                                                                # a list of available MODIS datasets
r = os.popen('cd ..; cd data/modis; ls -al *.nc').readlines()       # Opens all MODIS data saved in appropriate folder
#print (r[0])                                                       # Line for checking wether previous command works 
#print ('type r=', type(r[0]))                                      # confirms type of object saved as r
for q in r:                                                         # opens each file and changes filename for easier use
    b = q.split()[fnm_field_index].rstrip()                         # removes whitespace from files to make them more easily callable
    print('b =', b)                                                # Prints each datafile name to check for success of previous line
    m.append(xr.open_dataset('../data/modis/' + b))                 # Append the dataset with new, cleaned up filenames

print(len(m))                                                       # confirms amount of files saved in dataset

b = A20191532019160.L3m_8D_CHL_chlor_a_4km.nc
b = A20191612019168.L3m_8D_CHL_chlor_a_4km.nc
b = A20191692019176.L3m_8D_CHL_chlor_a_4km.nc
b = A20191772019184.L3m_8D_CHL_chlor_a_4km.nc
b = A20191852019192.L3m_8D_CHL_chlor_a_4km.nc
b = A20191932019200.L3m_8D_CHL_chlor_a_4km.nc
b = A20192012019208.L3m_8D_CHL_chlor_a_4km.nc
b = A20192092019216.L3m_8D_CHL_chlor_a_4km.nc
b = A20192172019224.L3m_8D_CHL_chlor_a_4km.nc
b = A20192252019232.L3m_8D_CHL_chlor_a_4km.nc
b = A20192332019240.L3m_8D_CHL_chlor_a_4km.nc
b = A20192412019248.L3m_8D_CHL_chlor_a_4km.nc
b = A20192492019256.L3m_8D_CHL_chlor_a_4km.nc
13


### Exploring data

Now that we've loaded our datasets we can explore them at our leisure.

In [5]:
# Specify the dataset you want to explore
ds = m[2]

# Print the selected dataset to see what dimensions, coordinates, variables, and attributes it contains.
print(ds)
print()

# Print desired attributes for easier reading
print("Time coverage start:", ds.attrs['time_coverage_start'])      # Confirm time of data collections for dataset

<xarray.Dataset>
Dimensions:  (eightbitcolor: 256, lat: 4320, lon: 8640, rgb: 3)
Coordinates:
  * lat      (lat) float32 89.979164 89.9375 89.89583 ... -89.93751 -89.97918
  * lon      (lon) float32 -179.97917 -179.9375 ... 179.93752 179.97917
Dimensions without coordinates: eightbitcolor, rgb
Data variables:
    chlor_a  (lat, lon) float32 ...
    palette  (rgb, eightbitcolor) uint8 ...
Attributes:
    product_name:                      A20191692019176.L3m_8D_CHL_chlor_a_4km.nc
    instrument:                        MODIS
    title:                             MODISA Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          Aqua
    temporal_range:                    8-day
    processing_version:                2018.0
    date_created:                      2019-08-06T01:56:59.000Z
    history:                           l3mapgen par=A20191692019176.L3m_8D_CH...
    l2_flag_names:                

## Plotting

### Map Plots

Now we can begin generating plots of our MODIS satellite chlorophyll data. Overall, we're focused on three of the OOI Regional Cabeled Array locations at the Oregon Coast, so we'll eventually want to create a map that includes these coordinates.

#### Interesting Locations and coordinates:

Site Name            |   Lat       |        Lon
------------------   |   ---       |        ---
Hawaii - USA         |  19.8968    |      -155.5828
Wedell Sea           |  -71.1879   |      -45.0000
Strait of Gibralter  |  35.9982    |      -5.6879

#### Things to improve on in this section:
* write code that allows users to specify central location and then plots an area of desired size around said location

For the zoomed out plots:

* add titles to plots so viewers can see what data range each map plot corresponds to. Entering `title=modis.attrs["time_coverage_start"]` into the `plot.contourf` line doesn't seem to work, nor does adding `modisplot.title("modis.attrs["time_coverage_start"])` on the next line. Simply typing `print (modis.attrs["time_coverage_start"]` in the plotting loop prints out all of the dates at once, before the plots have time to load.
* place plots into subfigure locations to make viewing easier
* add plot markers to indicate locations of RCA. Kept getting QuadMesh errors when trying to add markers before.


Start with a zoomed in plot of sea surface chlorophyll in the approximate location of the OOI Axial Base shallow profiler. The following cell is working with the previously selected dataset.
* location of axial base shallow profiler: -129.75333 45.83049 (from table above)
* Resolution of 0.1 degrees with automated colorbar

In [ ]:
#Select the desired location to plot
lats = 19.8968
lons = -155.5828
res = 1
s_lat1 = (lats - 1)
s_lat2 = (lats + 1)
s_lon1 = (lons - 1)
s_lon2 = (lons + 1)

modis = ds.sel(lat=slice(s_lat2,s_lat1), lon=slice(s_lon2,s_lon1))   

# Uncomment these to find the file name and details for the dataset you are plotting
# print(modis.product_name)
# print(modis.chlor_a)

# Set up a plot with a specified figure size and set lat/lon to appropriate axes
modisplot = modis.chlor_a.plot(x='lon', y='lat', figsize=(12,12))

Next, zoom out to look at the Oregon Coast region. White areas on the map represent missing data. The Orgeon coast can be seen on the right side of the map (colored white because there's no chlorophyll data on land). Cloud cover is responsible for white diamonds oversea and is varied across datasets. Some days have too much cloud cover to see significant trends, others have nearly no cloud cover and provide an excellent view.

Here, we're looking at a region that encompasses the three OOI RCA locations in the table above. Setting the colorbar to a high resolution with a relatively low max value elucidates the offshore eddies and swirls of chlorophyll.




In [ ]:
# Set colorbar values. May take trial and error to get the level of detail you are aiming for.
vmin = 0.0
vmax = 1.0

# Set levels for resolution of colorbar. Change the 0.01 value for higher or lower resolution. 
lvl = np.arange(vmin, vmax, 0.01).tolist()

# Create a plot for every open dataset
for ds in m:
    modis = ds.sel(lat=slice(47.5,44.4), lon=slice(-130,-123.5))    # Select location
    #print(modis.product_name)
    #print(modis.lat.values[0], modis.lon.values[0], modis.lat.values[-1], modis.lon.values[-1])
    #print(modis.chlor_a)
    modisplot = modis.chlor_a.plot.contourf(x='lon', y='lat', figsize=(12,12), vmin=0, vmax=1, levels=lvl)

### Combining datasets

Now that we've explored the individual datasets, we will combine them to get a timeseries of chlorophyll data. 

In [ ]:
# This preprocessor ensures that each new dataset is given a unique time value for its 
#   new time coordinate variable.

timeseries_timestamps = []

def preprocessor(ds):
    global timeseries_timestamps
    ds1 = ds.sel(lat=slice(47.5,44.4), lon=slice(-130,-123.5))    # selecting only the desired lat and lon speeds up data processing
    s = ds1.attrs['time_coverage_start']    # using time_coverage_start instead of date_created
    this_timestamp = pd.Timestamp(s)
    while this_timestamp in timeseries_timestamps: this_timestamp += timedelta(hours = 1)
    timeseries_timestamps.append(this_timestamp)
    ds1['time'] = xr.Variable('time', [this_timestamp])
    return ds1


# Combine all the open datasets into a multifile dataset along a new coordinate, time
# Also create a list of dates 
dates_list = []
for n in m:
    name = n.attrs['product_name']
    ds = xr.open_mfdataset('../data/modis/*.nc', combine='by_coords', preprocess = preprocessor)
    dates_list.append(n.attrs['time_coverage_start'])
print("Dates list:")
dates_list    # check that dates are chronological
print()

# View details of new combined dataset to check that location selection and combination along new coordinate worked
print("New multifile dataset:")
ds

### Time Series Plots

Now let's make time series plots of the chlorophyll a values collected by MODIS at three locations. Missing data is due to cloud coverage. 

Again, the locations and coordinates of our three points of interest:

Site                 |   Lat       |        Lon
------------------   |   ---       |        ---
Oregon Offshore      |  44.37415   |      -124.95648
Oregon Slope Base    |  44.52897   |      -125.38966
Axial Base           |  45.83049   |      -129.75326

Keep in mind that we are plotting using `method="nearest"` and the MODIS data may not contain data at the exact coordinates of the RCA locations. The satellite resolution is good enough that this should not be a major concern.

In [ ]:
# Create a time series plot for Oregon Offshore MODIS data

fig, axes = plt.subplots(figsize=(14,5))
axes.set(xlabel = 'Date', ylabel = 'Concentration (mg m^-3)', title = 'Chlorophyll over Summer 2019 at Oregon Offshore')
ds1 = ds.sel(lat=44.37415, lon=-124.95648, method="nearest")    # select location
axes.plot(ds1['time'], ds1.chlor_a, marker='o') 

print(ds1.coords)    # print the coordinates that were plotted

In [ ]:
# Get a table of [date | chlor] for Oregon Offshore
# This cell is much faster after adding location selection to the preprocessor function! 

ds1 = ds.sel(lat=44.37415, lon=-124.95648, method="nearest")
chlor = ds1.chlor_a.values
dates = ds1.time.values

for i in range(len(ds1.time)):
    print("date: ", ds1.time.values[i], " chlor: ", ds1.chlor_a.values[i])

In [ ]:
# Create a time series plot for Oregon Slope Base MODIS data

fig, axes = plt.subplots(figsize=(14,5))
axes.set(xlabel = 'Date', ylabel = 'Concentration (mg m^-3)', title = 'Chlorophyll over Summer 2019 at Oregon Slope Base')
ds1 = ds.sel(lat=44.52897, lon=-125.38966, method="nearest") # select location
axes.plot(ds1['time'], ds1.chlor_a, marker='o') 

print(ds1.coords)    # print the coordinates that were plotted

In [ ]:
# Get a table of [date | chlor] for Oregon Slope Base

ds1 = ds.sel(lat=44.52897, lon=-125.38966, method="nearest")
chlor = ds1.chlor_a.values
dates = ds1.time.values

for i in range(len(ds1.time)):
    print("date: ", ds1.time.values[i], "chlor: ", ds1.chlor_a.values[i])

In [ ]:
# Create a time series plot for Axial Base MODIS data

fig, axes = plt.subplots(figsize=(14,5))
axes.set(xlabel = 'Date', ylabel = 'Concentration (mg m^-3)', title = 'Chlorophyll over Summer 2019 at Axial Base')
ds1 = ds.sel(lat=45.83049, lon=-129.75326, method="nearest")
axes.plot(ds1['time'], ds1.chlor_a, marker='o') 

print(ds1.coords)    # print the coordinates that were plotted

In [ ]:
# Get a table of [date | chlor] for Axial Base MODIS data
# Note that there are more data points in the table below than the plot above would imply - fixed this by adding markers to plots

ds1 = ds.sel(lat=45.83049, lon=-129.75326, method="nearest")
chlor = ds1.chlor_a.values
dates = ds1.time.values

for i in range(len(ds1.time)):
    print("date: ", ds1.time.values[i], "chlor: ", ds1.chlor_a.values[i])

### intermezzo remarks

- The image saver below does not work properly; I just get an empty image...
- The nbextension command should *not* be necessary but it is for the record:
  
```
jupyter nbextension enable --py --sys-prefix ipyleaflet
```

- install the MODIS WMS from [here](https://lpdaac.usgs.gov/data_access/web_map_services_wms)

So basically lots to do here.

In [ ]:
# bug: This does not work properly yet

# plt.savefig(data + '/images/modis_SST.png')
import matplotlib
img=matplotlib.image.imread(image_d + 'modis_sst.png')
imgplot = plt.imshow(img)
# plt.show()

In [ ]:
# The goal is to overlay an image (created earlier; .png file) with a controllable opacity
# 
# How I got to opacity control through a sequence of print statements: 
#   print(thisMap)
#   print(thisMap.layers)
#   print(thisMap.layers[1])                    which is cheating by hardcoding the layer index as 1
#   print(thisMap.layers[1].opacity)
#   therefore setting thisMap.layers[1].opacity = 0.2 works just fine

def ChangeMapOpacity(opacity):thisMap.layers[1].opacity = opacity

from ipyleaflet import Map, ImageOverlay, WMSLayer
opacity = 0.7
thisMap = Map(center=(47, -129), zoom=5, layout=Layout(width='100%', height='600px'))
# The path to the image for overlay is relative; I could not get an absolute path to work...
sourceImage =  '../data/images/modis_sst.png'
image_layer = ImageOverlay(url=sourceImage, bounds=((40.2, -138.58), (53.8, -122.45)), opacity=opacity)
thisMap.add_layer(image_layer)


interact(ChangeMapOpacity, opacity = widgets.FloatSlider(min=0., max=1., step=0.025, value=opacity, 
      continuous_update=False, display='Opacity'))

# It seems that 'thisMap' must be the last line of code in the cell, hence placing interact() above
thisMap

# The following works but does not track zoom with new higher resolution tiles
# wms = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?",layers="nasa:bluemarble")
# thisMap.add_layer(wms)
    

In [ ]:
# This cell selects out MODIS-derived sea surface temperature in deg C at specific locations
#   This is for comparative work with in situ sensors in the Regional Cabled Observatory, which is
#   in turn part of the NSF-sponsored Ocean Observing Initiative

# OSB data coughs up 44.52897 -125.38966

eoLat, eoLon, eoDep          = 44. + 22./60. + 10./3600., -(124. + 57./60. + 15./3600.),  582.
osbLat, osbLon, osbDep       = 44. + 30./60. + 55./3600., -(125. + 23./60. + 23./3600.), 2906.
shrLat, shrLon, shrDep       = 44. + 34./60. +  9./3600., -(125  +  8./60. + 53./3600.),  778.
axbLat, axbLon, axbDep       = 45. + 49./60. +  5./3600., -(129. + 45./60. + 13./3600.), 2605.
ashesLat, ashesLon, ashesDep = 45. + 56./60. +  1./3600., -(130. +  0./60. + 50./3600.), 1543.
axcLat, axcLon, axcDep       = 45. + 57./60. + 17./3600., -(130. +  0./60. + 32./3600.), 1528.
axeLat, axeLon, axeDep       = 45. + 56./60. + 23./3600., -(129. + 58./60. + 27./3600.), 1516.
axiLat, axiLon, axiDep       = 45. + 53./60. + 35./3600., -(129. + 58./60. + 44./3600.), 1520.
ca = [('Endurance Offshore',                eoLat,    eoLon,    eoDep, 'Endurance Offshore'),
      ('Oregon Slope Base',                osbLat,   osbLon,   osbDep, 'Oregon Slope Base'),
      ('Southern Hydrate Ridge',           shrLat,   shrLon,   shrDep, 'Southern Hydrate Ridge'),
      ('Axial Base',                       axbLat,   axbLon,   axbDep, 'Axial Base'),
      ('Axial ASHES Vent Field',         ashesLat, ashesLon, ashesDep, 'Inferno vent'),
      ('Axial Caldera Center',             axcLat,   axcLon,   axcDep, 'Axial Caldera Center'),
      ('Axial Caldera East',               axeLat,   axeLon,   axeDep, 'Axial Caldera East'),
      ('Axial International Vent Field',   axiLat,   axiLon,   axiDep, 'Axial International Vent Field')
    ]

# for i in ca: print(i[0], i[1], i[2])
# for i in range(3): for j in range(3): print(m.MODISA_L3m_SST_2014_sst[0][i][j].values)
# 
# testLat = ca[1][1]       #   44.51528
# testLon = ca[1][2]       # -125.38972
# print(testLat, testLon)

sst = []
for i in range(len(ca)):
    # print(m.MODISA_L3m_SST_2014_sst.sel(lat=ca[i][1], lon=ca[i][2], method = 'nearest'))
    temp = modis.MODISA_L3m_SST_2014_sst.sel(lat=ca[i][1], lon=ca[i][2], method = 'nearest').values
    sst.append(round(float(temp[0]), 2))

# Here are MODIS-derived day-time sea surface temperatures at the cabled array locations on 2016-01-01T00:15:10 
print(sst)

In [ ]:
# And this works properly albeit with limited tiling resolution
from ipyleaflet import Map, WMSLayer
wms = WMSLayer(
url="https://demo.boundlessgeo.com/geoserver/ows?",
layers="nasa:bluemarble"
)
m = Map(layers=(wms, ), center=(42.5531, -48.6914), zoom=3)
m

In [ ]:
def pp(ds):
    ds['time'] = xr.Variable('time', [pd.Timestamp(ds.attrs['time_coverage_start']) + pd.Timedelta(15, unit='d')])
    return ds

cb=xr.open_mfdataset('modis_chlora_2017_*.nc', preprocess = pp, concat_dim='time').chlor_a.to_dataset()
# cb.chlor_a shows units as mg / cubic meter, numerically equivalent to ug / liter

p,a=plt.subplots(figsize=(13,9))

for x in np.arange(-125.38966-0.1, -125.38966+0.1, 0.1):
    for y in np.arange(44.52897-0.1, 44.52897+0.1, 0.1):
        a.plot(cb.time.values, cb.chlor_a.sel(lon=x, lat=y, method='nearest').values, 'o-')

In [ ]:
# cb.to_netcdf('modis_chlora_5_month_timeseries.nc')

In [ ]:
cb

In [ ]:
latselect = np.arange(44.52897-1., 44.52897+1.00001, 0.10)
lonselect = np.arange(-125.38966-1., -125.38966+1.00001, 0.10)
local = cb.sel(lat = latselect, lon = lonselect, method = 'nearest')
local

In [ ]:
%%time
p,a=plt.subplots(figsize=(13,9))
for i in range(9,12):
    for j in range(9,12):
        a.plot(local.time.values, local.chlor_a.isel(lat=i,lon=j).values, 'o-')

a.plot(local.time.values, local.chlor_a.isel(lat=10,lon=10).values, 'o-', linewidth=6)

In [ ]:
local.to_netcdf('local.nc')

In [ ]:
local=xr.open_dataset('local.nc')
local

In [ ]:
l1

In [ ]:
# p,a=plt.subplots(figsize=(13,9))
# for t in l1.chlor_a: print(t)  
# a.plot(local.time.values, t.values, 'o-')

for i in range(21): 
    for j in range(21): 
        print(l1.chlor_a[0:5, i, j].values)

In [ ]:
local = cb.reindex_like()

In [ ]:
local.chlor_a[0].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

In [ ]:
local.chlor_a[1].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

In [ ]:
local.chlor_a[2].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

In [ ]:
local.chlor_a[3].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

In [ ]:
local.chlor_a[4].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

In [ ]:
ca=xr.open_rasterio('./chlora.tif').to_dataset(name='chlora')
ca

In [ ]:
ca.chlora

In [ ]:
cad=ca.where(ca.chlora<1.4)
cad.chlora.plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=1.0)
plt.xlim(-128,-124)
plt.ylim(43.5,45.5)
plt.plot(-125.38966, 44.52897, '^')
# a=p.axes()
# a.set(xlim=(-155,-135))

In [ ]:
float(ca.sel(band=0,y=44.52897,x=-125.38966, method='nearest').values)

In [ ]:
for x in np.arange(-125.5, -125.0, 0.1):
    for y in np.arange(44.3, 44.8, 0.1):
        print(float(ca.sel(band=0,y=y,x=x, method='nearest').values))